# Tools and Programming Languages for Data Science: Project 2

### Notes: 

- I will use the abbreviations df/dfs for dataframe/s in this Notebook.
- I will use the term "sub df" (=subset dataframe) for dataframes which I sliced out from another dataframe.
- For exercise 1 and 2 it was necessary to get a first overview or some extra data exploration of all datasets/combined dataframes and to discover entries, values, dtypes etc. that needed to be fixed. I did this in a seperate notebook and don't show it here, but I explain my findings.

In [57]:
import pandas as pd
import requests
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

# Exercise 1

I decided that my function should take three lists for the arguments. 
For the "indicators" argument a list must be provided and the elements must be strings. The same applies for the "countries" argument. The "years" list can just contain one single year or two years if a time range should be evaluated. In the latter case the smaller/earlier year number will be used as the beginning of the range and the greater/later year number as the end, even if someone would provide it the opposite way. Furthermore it doesn't matter if the years within the list are provided as ints, floats or strings. In any case my function transforms it in a way, that it can be used for the API call. Also, if some argument isn't provided the right way, my function prints an error message with a hint how the respective argument should be typed in.

Generally my function iterates over the indicators list. For every indicator my function uses all provided countries and all provided years to make one API call. But as the API just returns the 50 items per page by default, there could be many cases where my function would have to make multiple requests (one for each page). Therefore, when the function made the first call for an indicator it checks the total number of items in the response. If the total number of items is <= 50, all items are one the first page and no modification has to be made. Otherwise the function makes a new API call, but now setting the "per_page" argument in the URL with the total number of items. Then we would get all items on one page. So there are AT MOST two calls for each indicator: one if total number of items <= 50 or two if total number of items > 50. Also responses will always be given in json format, as I set the parameter this way.

From the responses the function will collect certain information which will be stored in lists. These lists will be used at the end to create entries for the df. Each entry/row will contain the country name, the specific year, indicator ID, indicator in written format (indicator IDs aren't really self-explanatory) and the realized value of the indicator.

In [58]:
def get_worldbank_data(indicators, countries, years):
    """
    Query data from the Worldbannk Indicator API. For each indicator all provided countries for all provided years will be evaluated.
    
    :param list of str indicators: List of indicators to search. 
                                   Each list element has to be type str.
    :param list of str countries:  List of the countries two-letter ISO codes to search for each indicator. 
                                   Alternatively "all" for all countries.
                                   Each list element has to be type str.
    :param list of int/str years:  Year(s) to search for each country and each indicator. 
                                   Either one single year or two years for range, with first year = start, second year = end.
                                   Each list element has to be type int or str.

    :return: Data Frame with requested data
    :rtype: pandas.core.frame.DataFrame.
    """

    if len(countries) >= 1:                                    # at least one county must be given
        countries_for_URL = ";".join(countries)                # join country/countries for putting them into the request URL
    else:
        print("Error: Please provide at least one country or 'all' for the API request.")

    if (len(years) == 1) | (len(years) == 2):                  # one year must be given or two years for a time range
        years = [int(float(year)) for year in years]           # no matter how elements in years are given initially-> convert elements to int
        years.sort(key=int)                                    # sort year(s)
        years_for_URL = ":".join(str(year) for year in years)  # join year(s) for putting them into the request URL
    else:
        print("Error: Please provide a single year or two years for range, with first year = start, second year = end.")

    if len(indicators) >= 1:                                   # at least one county must be given
        country_name_lst, year_lst, indicator_id_lst, indicator_lst, value_lst = [], [], [], [], []   # lists to collect information for df
        
        for indicator in indicators:
            indicator_for_URL = indicator                      # indicator for request URL
            URL = f"https://api.worldbank.org/v2/country/{countries_for_URL}/indicator/{indicator_for_URL}?date={years_for_URL}&format=json"
            response = requests.get(URL)
            data = response.json()
            
            if data[0]["total"] > 50:                          # check if there are more than 50 items -> if true, new API call
                results_per_page = data[0]["total"]            # for new API call: get all items on one page
                URL = f"https://api.worldbank.org/v2/country/{countries_for_URL}/indicator/{indicator_for_URL}?" +\
                      f"date={years_for_URL}&per_page={results_per_page}&format=json"
                response = requests.get(URL)
                data = response.json()

            for dict in data[1]:                                    # iterate over all dictionaries/entries in the API response
                country_name_lst.append(dict["country"]["value"])   # append list with country name
                year_lst.append(dict["date"])                       # append list with year 
                indicator_id_lst.append(dict["indicator"]["id"])    # append list with indicator id
                indicator_lst.append(dict["indicator"]["value"])    # append list with indicator in written format
                value_lst.append(dict["value"])                     # append list with realized indicator value
    else:
        print("Error: Please provide at least one indicator for the API request.")
    
    # after collecting and storing all infos in lists -> create df:
    final_df = pd.DataFrame({"country_name": country_name_lst,     
                            "year": year_lst,
                            "indicator_id": indicator_id_lst,
                            "indicator": indicator_lst,
                            "indicator_value": value_lst})
    
    return final_df 


Let's demonstrate that the function works with the following queries. Note how I provide the year values, to show that the function is relatively robust against "unconventional" inputs. You can also try to swap the years, e.g. for a) you can put in [2020, 2015] instead of [2015, 2020]. The result will still be the same, as the function order the years before it makes an API call.

### 1a) 
total population (SP.POP.TOTL) of Germany (DE) and France (FR) between 2015 and 2020:

In [59]:
pop_de_fr_2015_2020 = get_worldbank_data(["SP.POP.TOTL"], ["DE","FR"], ["2015.0", "2020.0"])
pop_de_fr_2015_2020

,country_name,year,indicator_id,indicator,indicator_value
0,Germany,2020,SP.POP.TOTL,"Population, total",83160871
1,Germany,2019,SP.POP.TOTL,"Population, total",83092962
2,Germany,2018,SP.POP.TOTL,"Population, total",82905782
3,Germany,2017,SP.POP.TOTL,"Population, total",82657002
4,Germany,2016,SP.POP.TOTL,"Population, total",82348669
5,Germany,2015,SP.POP.TOTL,"Population, total",81686611
6,France,2020,SP.POP.TOTL,"Population, total",67571107
7,France,2019,SP.POP.TOTL,"Population, total",67388001
8,France,2018,SP.POP.TOTL,"Population, total",67158348
9,France,2017,SP.POP.TOTL,"Population, total",66918020


### 1b) 
total population (SP.POP.TOTL), GDP in current US$ (NY.GDP.MKTP.CD) and life expectancy in years at birth (SP.DYN.LE00.IN) of all countries in 2012 (print shape of resulting DataFrame and display first 10 rows):

In [60]:
pop_gdp_lifeexp_all_2012 = get_worldbank_data(["SP.POP.TOTL", "NY.GDP.MKTP.CD", "SP.DYN.LE00.IN"], ["all"], ["2012"])
num_rows = pop_gdp_lifeexp_all_2012.shape[0]
num_col = pop_gdp_lifeexp_all_2012.shape[1]
print("Shape: ", pop_gdp_lifeexp_all_2012.shape)
print("The resulting data frame has", num_rows, "rows and", num_col, "columns." )
pop_gdp_lifeexp_all_2012.head(10)


Shape:  (798, 5)
The resulting data frame has 798 rows and 5 columns.


,country_name,year,indicator_id,indicator,indicator_value
0,Africa Eastern and Southern,2012,SP.POP.TOTL,"Population, total",5.525307e+08
1,Africa Western and Central,2012,SP.POP.TOTL,"Population, total",3.767980e+08
2,Arab World,2012,SP.POP.TOTL,"Population, total",3.803834e+08
3,Caribbean small states,2012,SP.POP.TOTL,"Population, total",2.898802e+06
4,Central Europe and the Baltics,2012,SP.POP.TOTL,"Population, total",1.039353e+08
5,Early-demographic dividend,2012,SP.POP.TOTL,"Population, total",3.035160e+09
6,East Asia & Pacific,2012,SP.POP.TOTL,"Population, total",2.243777e+09
7,East Asia & Pacific (excluding high income),2012,SP.POP.TOTL,"Population, total",2.001246e+09
8,East Asia & Pacific (IDA & IBRD countries),2012,SP.POP.TOTL,"Population, total",1.976387e+09
9,Euro area,2012,SP.POP.TOTL,"Population, total",3.404507e+08


### 1c) 
How many API calls makes your function for a) and b) respectively?

### Answer: 

The number of API calls depends on the number of elements in the provided "indicators" list. For each indicator my function makes either one API call, if the number of total elements in the response is <= 50, or 2 API calls, if the number of total elements in the response is > 50. 

Therefore for a) my function makes one API call. For b) it makes 6 API calls, because there are 3 indicators and for each indicator the function has to make a second API call after the first one, as for each indicator the response contains 266 total elements. 

Imagine the function would not check the total number of items and therefore just read every page of the initial response. Then every page would have 50 items at most, but to get the next page it would need a new API call. For b) this would lead to 6 API calls per indicator, resulting in 18 calls overall, which wouldn't be efficient at all.

# Exercise 2

### 2a)

First I read in the medal table csv and store in the variable "medal_table". As i want to join the dfs "medal_table" and "pop_gdp_lifeexp_all_2012" via the country names, I need the country names to match exactly. So I checked which countries from the medal table are not found in the "country_name" column of the "pop_gdp_lifeexp_all_2012" df. Indeed there were some countries which couldn't be found, however not because they were really missing but just because they had a different name in "pop_gdp_lifeexp_all_2012". I decided to replace these country names in the medal table with the country names as they are specified in "pop_gdp_lifeexp_all_2012", because the df "medal_table" is smaller. Unfortunately there was one country from the medal table which really isn't included in the other df and that's Chinese Taipei (better kown as Taiwan). After some research I found out that the Worldbank doesn't collect information about Taiwan and it's also not included in the values for China but just for some worldwide evaluatios. After all I decided to drop Chinese Taipei/Taiwan from the medal table, because it doesn't have any usecase for the follwoing tasks if there is no data in the Worldbank df. In the end I add a column with the total number of medals (sum up gold, silver and bronze) for each country which will be used in exercise 3:

In [61]:
medal_table = pd.read_csv("medal_table.csv", header=0)     # read medal table CSV

# replace country names in "medal_table" with corresponding country names from "pop_gdp_lifeexp_all_2012":
medal_table.replace({"People's Republic of China": "China",
                     "Great Britain": "United Kingdom",
                     "Republic of Korea": "Korea, Rep.",
                     "Islamic Republic of Iran": "Iran, Islamic Rep.",
                     "Czech Republic": "Czechia",
                     "Democratic People's Republic of Korea": "Korea, Dem. People's Rep.",
                     "Turkey": "Turkiye",
                     "The Bahamas": "Bahamas, The",
                     "Venezuela": "Venezuela, RB",
                     "Egypt": "Egypt, Arab Rep.",
                     "Slovakia": "Slovak Republic",
                     "Hong Kong, China": "Hong Kong SAR, China",
                     "Kingdom of Saudi Arabia": "Saudi Arabia",
                     "Vietnam": "Viet Nam"},inplace=True)

medal_table.drop(medal_table[medal_table["country"]=="Chinese Taipei"].index, inplace=True)    # drop country Chinese Taipei/Taiwan
medal_table["sum_medals"] = medal_table[["gold", "silver", "bronze"]].sum(axis=1)              # add column with total nuber of medals
medal_table.reset_index(inplace=True, drop=True)                                               # reset index

Now I merge "medal_table" with "pop_gdp_lifeexp_all_2012" via the country names. I use left_on and right_on as the columns have slightly different names. Also there are way more countries in "pop_gdp_lifeexp_all_2012" than in "medal_table, because many countries didn't win a medal in 2012. For the following exercises I just need the countries which also appear in the medal table, so I use an inner join for the merge. The resulting df will be the base for the upcoming tasks and therefore I store it in a new variable called "medals_all_indicators_df". Further I drop some columns which aren't useful after the merge and rename one column which contains the year. 

The next step is to pivot on the "indicator" column. If I wouldn't do this the df would look like the ones in 1a) and 1b), with an column "indicator" and an extra column "indicator_value". I want the indicators to be the column headers and fill these columns with the values for the respective coutry. 

At the end I do some final cleaning, including renaming the indicator columns and sorting the df by countries in alphabetical order. Even if it wasn't explicitly required, I print the first rows to show how the final result "medals_all_indicators_df" looks like:

In [62]:
medals_all_indicators_df = pd.merge(left= medal_table, right= pop_gdp_lifeexp_all_2012,          # merge "medal_table" with "pop_gdp_lifeexp_all_2012"
                                left_on="country", right_on="country_name", how="inner")    
medals_all_indicators_df.drop(columns=["country_name", "year_y", "indicator_id"], inplace=True)  # drop columns
medals_all_indicators_df.rename({"year_x":"year"}, axis=1, inplace=True)                         # rename year column

# now pivot the df, to get a cleaner look:
medals_all_indicators_df = medals_all_indicators_df.pivot(index=["year", "country", "country_code", "gold", "silver", "bronze", "sum_medals"],
                                                          columns="indicator",
                                                          values="indicator_value")

# final cleaning:
medals_all_indicators_df.reset_index(inplace=True)                          # reset index
medals_all_indicators_df.columns.name = None                                # remove name of columns index 
medals_all_indicators_df.rename(columns={"country_code": "ISO_3_code",      # rename columns
                                         "GDP (current US$)": "GDP_in_USD",                      
                                         "Life expectancy at birth, total (years)": "life_expect_in_years",
                                         "Population, total": "population"}, inplace=True)
medals_all_indicators_df["population"] = medals_all_indicators_df["population"].astype(int)    # convert population numbers to integers
medals_all_indicators_df.sort_values(by="country", inplace=True)                               # sort df by country names
medals_all_indicators_df.head(4)         

,year,country,ISO_3_code,gold,silver,bronze,sum_medals,GDP_in_USD,life_expect_in_years,population
0,2012,Afghanistan,AFG,0,0,1,1,1.990733e+10,61.923,30466479
1,2012,Algeria,ALG,1,0,0,1,2.271437e+11,74.202,37260563
2,2012,Argentina,ARG,1,1,2,4,5.459824e+11,76.467,41733271
3,2012,Armenia,ARM,0,1,1,2,1.061932e+10,73.454,2914421


# IMPORTANT NOTE:

Ich checked my final "medals_all_indicators_df" df for missing values and indeed there is one row (Nort Korea) which doesn't have data in the GDP column. As this doesn't affect task 2b) I will keep this row for now but then I will drop it at the beginning of exercise 3.

### 2b)

For this task I create a copy of "medals_all_indicators_df" and store it in "medals_population_df", so I can manipulate it without affecting the original df. 

The column "population" contains the population number for all countries. I divide this column by 10 000 000 to get the population number in 10 million units and assign the result to a new column "pop_in_10_millions". In want to show this new column in the alternative medal table too because in my opinion it helps in assessing the results. Next I divide the "gold", "silver" and "bronze" columns by the new "pop_in_10_millions" column to calculate the number of medals per 10 million inhabitants for each country. First I thought about rounding the results to full integers but finally I decided not to do it, because I think we would lose some information as we divided the initial medal counts by 10 million inhabitants.

Furthermore I drop several columns which aren't necessary for the final representation. Finally I sort the df by gold, silver and bronze (all per 10 million inhabitants), insert a colum with the new ranks and show you the first ten countries. As now there is the "new_rank" column, I don't show the index of the df:

In [63]:
medals_population_df = medals_all_indicators_df.copy()                                            # create copy
medals_population_df["pop_in_10_millions"] = medals_population_df["population"].div(10000000)     # population in 10 mil units
medals_population_df[["gold/10_mil_inhab", "silver/10_mil_inhab", "bronze/10_mil_inhab"]] = \
    medals_population_df[["gold", "silver", "bronze"]].div(medals_population_df["pop_in_10_millions"], axis=0)  # number medals per 10 mil inhabitants

medals_population_df.drop(columns=["gold", "silver", "bronze", "sum_medals", "GDP_in_USD", "life_expect_in_years"], inplace=True)       # drop unnecessary columns 
medals_population_df.sort_values(by=["gold/10_mil_inhab", "silver/10_mil_inhab", "bronze/10_mil_inhab"], inplace=True, ascending=False) # sort df by medals
medals_population_df.reset_index(inplace=True, drop=True)                                                          # reset index after new order                                         
medals_population_df.insert(1, column="new_rank", value=list(range(1, len(medals_population_df)+1)))               # insert column with new ranks
medals_population_df["population"] = medals_population_df.apply(lambda x: "{:,}".format(x["population"]), axis=1)  # separate 1000 units with comma
medals_population_df.head(10).style.hide()

year,new_rank,country,ISO_3_code,population,pop_in_10_millions,gold/10_mil_inhab,silver/10_mil_inhab,bronze/10_mil_inhab
2012,1,Grenada,GRN,"115,912",0.011591,86.272345,0.000000,0.000000
2012,2,"Bahamas, The",BAH,"382,061",0.038206,26.173831,0.000000,0.000000
2012,3,Jamaica,JAM,"2,759,817",0.275982,14.493715,18.117143,10.870286
2012,4,New Zealand,NZL,"4,408,100",0.440810,13.611306,4.537102,11.342755
2012,5,Bahrain,BRN,"1,224,939",0.122494,8.163672,0.000000,0.000000
2012,6,Hungary,HUN,"9,920,362",0.992036,8.064222,4.032111,6.048166
2012,7,Croatia,CRO,"4,267,558",0.426756,7.029781,2.343260,4.686521
2012,8,Trinidad and Tobago,TTO,"1,430,377",0.143038,6.991164,6.991164,13.982328
2012,9,Lithuania,LTU,"2,987,773",0.298777,6.693949,0.000000,10.040923
2012,10,Latvia,LAT,"2,034,319",0.203432,4.915650,0.000000,4.915650


# Exercise 3

As stated at the end of task 2a), now I drop the row (North Korea) which has a missing value in the GDP column:

In [64]:
medals_all_indicators_df.dropna(axis=0, how="any", inplace=True)
medals_all_indicators_df.reset_index(inplace=True, drop=True)

### 3a)

For this subtask I just follow the steps we discussed in the lecture. I define the columns "GDP_in_USD", "life_expect_in_years", "population" as features and the column "sum_medals" as target:

In [65]:
# define features and target for machine learning model:
X = medals_all_indicators_df[["GDP_in_USD", "life_expect_in_years", "population"]]     # features: GDP, life expectancy and  population
y = medals_all_indicators_df["sum_medals"]                                             # target: total number of medals

Then I use train_test_split() to split the data into training and test sets. After that I define my model as linear regression and train it with the training set by the fit-method(). By doing this, the model tries to find the best fitting line through the data, which is the line that minimizes the mean squared error. Finally I want to evaluate how good the model is. Therefore I use the X_test data to make predictions about the outcomes. Comparing these outcomes with the y_test data (which are the TRUE amounts of medals won by countries) will show how well the model fits. The metric for the comparison is the root mean squared error, which tells how far off the predicitons are on average from the true values:

In [66]:
# 1. split data into a training and a test set:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1) 

# 2. Train a linear regression model using population, life expectancy and the GDP per capita of a country as features:
model = LinearRegression()
model.fit(X_train, y_train)

# 3. evaluate the model using the root mean squared error as the performance metric:
y_pred = model.predict(X_test)                    # predict on the test set
rmse = root_mean_squared_error(y_test, y_pred)    # evaluate the model with the root mean squared error
print(f"Root Mean Squared Error (rounded): {round(rmse, ndigits=3)}")

Root Mean Squared Error (rounded): 5.579


### 3b)

Question: How do you judge the performance? 

Answer:

The root mean squared error measures the average distance of the data points from the regression line. As my model's RMSE is 5.579, it means that on average my model predicts that a country has 5.579 more or less medals than this country would have in reality. Considering that in 2012 the USA had the most medals with 104 in total, this means my model is around 5 % away from the truth (always remembering that rmse is an average distance). This would be kind of okay if all countries would have won that amound of medals. But the problem is that our df contains 84 countries and out of these there are just 5 which won more than 40 medals in total. Therefore the average error is quite high for the majority of countries.

Question: What are possible reasons for this performance?

Answer:

The reason for this performance might be the fact that the RMSE is prone to outliers (e.g. USA). Also there might be some kind of multicollinearity, which means that two or more independent variables have a strong correlation with each other. Multicollinearity can lead to a higher RMSE, so that might be an issue here. Another aspect could be that a linear regression model is just not the best model in this case and we should use some other kind of regression (logistic, polynomial, ridge regression).

Question: How could the model be improved?

Answer:

I already mentioned the proneness of the RMSE to outliers, so maybe we should exclude these outliers when we train our model. Someone could argue that we might use more independent variables for improvement. And that's not even wrong, because more independet variables reduce the RMSE value in ANY case, even if it doesn't make any sense. Considering this we should think about using independent variables that could really impact the overall performance of a country in the olympics. So the quality (or sense) of the indipendent variables should be more important than the quantity. For example we could replace the GDP with another monetary value, for example the amount of money a countrie government puts into the development of professional sports infrastructure. And/or for every country we could try to find out how many people are active in sports in general. Maybe societies with a higher percentage of people that do some kind of sports also perform better in the olympics. This metric could be more fitting than just using the total population number.

### 3c)

First I have to calculate the GDP of our hypothetical country with the given GDP per capita and total population values:

GDP_in_USD = GDP per capita * population = 20.000 US$/capita * 10,000,000 capitas = $200 * 10^9$ US$

Then I let my model predict the number of medals the hypothetical country would win. For the the output I round the result as a country can just win "full" medals:

In [67]:
test_country = pd.DataFrame([{"GDP_in_USD": 200 * 10**9, "life_expect_in_years": 70, "population": 10**7}]) # create our hypothetical country
print(f"The model predicts that the country would win {round(model.predict(test_country)[0])} medals.")

The model predicts that the country would win 6 medals.
